In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

pd.options.display.max_rows = 100

In [ ]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
print(test.head())

In [ ]:
# split training data into predictors and price (output)
x_train = train.iloc[:, 1:-1]
y_train = train.iloc[:, -1]
x_train

In [ ]:
test.drop(labels='Id', axis=1, inplace=True)
test

In [ ]:
df = pd.concat([x_train, test])
df.reset_index(drop=True, inplace=True)

In [ ]:
df.columns

In [ ]:
df.dtypes.head(80)

##### Subclass should be a qualitative variable, and needs to be changed to an object

In [ ]:
df['MSSubClass'] = df.MSSubClass.astype(str)
print(df.dtypes)

# Address Missing Values

In [ ]:
df.isna().sum().sort_values(ascending=False).head(35)

##### Almost all values in the Pool Quality/ Condition feature are missing, therefore feature will be dropped (pool area can be indicative of whether house has a pool)

In [ ]:
print(df.MiscFeature.value_counts())

##### The miscellaneous features indicate additional features where relevant, so can keep in the dataset

### Assign numeric ratings to columns with ranked values

In [ ]:
# Check if 'TA' is in the dataframe column. We'll take the columns in which evaluations exist and map them to ratings
cols = []

for column in df.columns:
    if 'TA' in df[column].values:
        cols.append(column)

# Assign ratings to the relevant columns

ratings_map = {np.nan: 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
df[cols] = df[cols].replace(ratings_map)
df

### Categorical Missing Value Imputation

In [ ]:
cat_nulls = df.loc[:, df.isna().sum() > 0].select_dtypes("object")
cat_nulls.isna().sum().sort_values(ascending=False)

In [ ]:
# In context, null values provide information about the dwelling, so null values do not need to be removed. For the missing zoning values, we can fill missing values by taking the most common land zone in the neighborhood
def mode_fill(data, feature, comparison_feature):
    data[feature] = data.apply(lambda row: data[data[comparison_feature] == row[comparison_feature]][feature].mode().iat[0] if pd.isna(row[feature]) else row[feature] if not data[data[comparison_feature] == row[comparison_feature]][feature].mode().empty else row[feature], axis=1)

In [ ]:
elec_chi2 = pd.DataFrame(index=cat_nulls.loc[:, cat_nulls.columns != 'Electrical'].columns, columns=['chi2', 'pval'])

for feature in cat_nulls.loc[:, cat_nulls.columns != 'Electrical'].columns:
    crosstab = pd.crosstab(cat_nulls.Electrical, cat_nulls[feature])
    chi2, pval, dof, expected = scipy.stats.chi2_contingency(crosstab)
    elec_chi2.loc[feature, 'chi2'] = chi2
    elec_chi2.loc[feature, 'pval'] = pval
    
elec_chi2.sort_values(by='chi2', ascending=False)

In [ ]:
utilities_chi2 = pd.DataFrame(index=cat_nulls.loc[:, cat_nulls.columns != 'Utilities'].columns, columns=['chi2', 'pval'])

for feature in cat_nulls.loc[:, cat_nulls.columns != 'Utilities'].columns:
    crosstab = pd.crosstab(cat_nulls.Utilities, cat_nulls[feature])
    chi2, pval, dof, expected = scipy.stats.chi2_contingency(crosstab)
    utilities_chi2.loc[feature, 'chi2'] = chi2
    utilities_chi2.loc[feature, 'pval'] = pval
    
utilities_chi2.sort_values(by='chi2', ascending=False)

In [ ]:
mode_fill(df, 'MSZoning', 'Neighborhood')
mode_fill(df, 'Electrical', 'Exterior1st')
mode_fill(df, 'Utilities', 'Electrical')

For the remaining categorical features, we can fill the null values to identify them as "Not Applicable", as it is reflective of a household feature not existing

In [ ]:
cat_nulls = cat_nulls.fillna("Not Applicable")
for col in cat_nulls:
    df[col] = cat_nulls[col]
    
cat_nulls.isna().sum()

### Numeric Missing Value Imputation

In [ ]:
df.isna().sum().sort_values(ascending=False).head(15)

##### Lot frontage should apply to most residencies, and therefore we'll use a regression analysis to determine this. To do this, we'll first plot the lot frontage values against the lot areas

In [ ]:
sns.scatterplot(data=df, x=df[df.LotFrontage.isna() == False].LotArea, y=df[df.LotFrontage.isna() == False].LotFrontage, alpha=0.4)
plt.xlim(0, 50000)
plt.show()
plt.clf()

print(f"Correlation between frontage and area: {scipy.stats.pearsonr(df[df.LotFrontage.isna() == False].LotArea, df[df.LotFrontage.isna() == False].LotFrontage)[0]}")
print(f"P-val of correlation between frontage and area: {scipy.stats.pearsonr(df[df.LotFrontage.isna() == False].LotArea, df[df.LotFrontage.isna() == False].LotFrontage)[1]}")

In [ ]:
lot_reg = LinearRegression()
lot_reg.fit(np.array(df[df.LotFrontage.isna() == False].LotArea).reshape(-1, 1), np.array(df[df.LotFrontage.isna() == False].LotFrontage).reshape(-1, 1))
frontage_pred = lot_reg.predict(np.array(df[df.LotFrontage.isna() == True].LotArea).reshape(-1, 1))

count = 0
frontage_pred = frontage_pred.flatten().tolist()

for index, row in df.iterrows():
    if pd.isna(row['LotFrontage']):
        df.loc[index, 'LotFrontage'] = frontage_pred[count]
        count += 1

print(df.LotFrontage.isna().sum())

##### For the remaining missing values, we can fill the values to be 0

In [ ]:
df = df.fillna(0)
df.isna().sum().sort_values(ascending=False)

# Feature Engineering

##### Plot distributions for numerical features, to check for any standardization required

In [ ]:
skew_df = pd.DataFrame(index=df.select_dtypes(np.number).columns, columns=['skewness'])

for count, feature in enumerate(df.select_dtypes(np.number).columns):
    skew_df.iloc[count, 0] = abs(scipy.stats.skew(df[feature]))

skew_df = skew_df[skew_df['skewness'] > 1].sort_values(by='skewness', ascending=False)
skew_df

In [ ]:
# Check the distribution of highly skewed features

count = 1
fig = plt.figure(figsize=(15, 12))

for feature in skew_df.index:
    plt.subplot(8, 3, count)
    sns.histplot(data=df, x=feature, kde=True)
    plt.title(f"{feature} distribution")
    
    count += 1

plt.tight_layout()
plt.show()
plt.clf()

In [ ]:
# Apply yeo johnson transformation for each feature
transformed_df = pd.DataFrame(columns=skew_df.index)
fig = plt.figure(figsize=(15, 12))
count = 1

for i in transformed_df.columns:
    transformed_df[i], lambda_ = scipy.stats.yeojohnson(df[i])

    plt.subplot(8, 3, count)
    sns.histplot(data=transformed_df, x=i, kde=True)
    plt.title(f"{i} transformed distribution")
    
    count += 1
    
    df[i] = transformed_df[i]
    

plt.tight_layout()
plt.show()
plt.clf()

##### Draw a heatmap for numeric feature correlation

In [ ]:
f,ax = plt.subplots(figsize=(25,25))
plt.title('Correlation Matrix')
sns.heatmap(df.select_dtypes(np.number).corr(method='pearson').corr(method='pearson'), annot=True, fmt = '.2f', ax=ax, vmin=-1, cmap='coolwarm_r')
plt.show()

# Transform remaining categorical variables

##### Since one-hot encoding does not do any favours with respect to dimensionality, target encoding will be used to encode categorical variables

In [ ]:
def cat_encode(data, target, col):
    trained = pd.concat([data.iloc[0:1460, :], target], axis=1)
    grouped = trained.groupby(by=col).agg({'SalePrice': 'mean'}).reset_index()
    grouped = grouped.sort_values(by=grouped.columns[1], ascending=True)
    
    grouped_dict = {col: {row.iloc[0]: row.iloc[1] for index, row in grouped.iterrows()}}
    print(grouped_dict)
    plt.figure(figsize=(20, 6))
    plt.subplot(1, 2, 1)
    sns.countplot(data=data, y=col)
    plt.title(f"{col} countplot")
    
    data[col] = data[col].map(grouped_dict[col])
    
    plt.subplot(1, 2, 2)
    sns.lineplot(data=grouped, x=grouped.iloc[:, 0], y=grouped.iloc[:, 1])
    plt.title(f"{col} distribution")
    
    plt.show()

In [ ]:
df2 = df.copy(deep=True)
for column in df2.select_dtypes("object"):
    cat_encode(df2, y_train, column)

##### Scale the data

In [ ]:
scaler = StandardScaler()
df3 = pd.DataFrame(scaler.fit_transform(df2), columns=df2.columns)
df3

In [ ]:
x_train = df3.iloc[0:1460, :]
x_test = df3.iloc[1460:, :]

# Target Transformation 

In [ ]:
sns.histplot(y_train, kde=True)
plt.show()

##### To make the distribution normal, we'll use log tranformation. First check that there are no 0's in the target feature, as log transformation cannot be applied to these

In [ ]:
print(min(y_train))

In [ ]:
y_train_new = np.log(y_train)

sns.histplot(y_train_new, kde=True)
plt.show()

In [ ]:
class ml_modelling:
    def __init__(self, x_train, y_train, x_test):
        self.x = x_train
        self.y = y_train
        self.x_test = x_test
        self.dict = {}
        
    def linear(self, components):
        mean = self.x.mean(axis=0)
        sttd = self.x.std(axis=0)
        data_standardized = (self.x - mean) / sttd
        
        pca = PCA(n_components=components)
        pca_x = pca.fit_transform(data_standardized)
        
        lr = LinearRegression()
        lr_scores = -cross_val_score(lr, pca_x, self.y, cv=5, scoring='neg_mean_squared_error')
        lr_score = lr_scores.mean()
        self.dict['Linear Regression'] = lr_score
        
        self.pca_x = pca_x
        self.lr = lr
        
        return lr_score
    
    def random_forest(self):
        rfr = RandomForestRegressor(self.x, self.y)
        rfr_scores = -cross_val_score(rfr, self.x, self.y, cv=5, scoring='neg_mean_squared_error')
        rfr_score = rfr_scores.mean()
        self.dict['Random Forest'] = rfr_score
                
        self.rfr = rfr
        
        return rfr_score
    
    def xgb(self):
        xgb = XGBRegressor()
        xgb_scores = -cross_val_score(xgb, self.x, self.y, cv=5, scoring='neg_mean_squared_error')
        xgb_score = xgb_scores.mean()
        self.dict['XG Boost'] = xgb_score
        
        self.xgb = xgb
        
        return xgb_score
    
    def lgbm(self):
        lgbm = LGBMRegressor()
        lgbm_scores = -cross_val_score(lgbm, self.x, self.y, cv=5, scoring='neg_mean_squared_error')
        lgbm_score = lgbm_scores.mean()
        self.dict['Light GBM'] = lgbm_score
        
        self.lgbm = lgbm
        
        return lgbm_score
    
    def ensemble(self):
        # Generate predictions from base models
        models = [self.lr, self.rfr, self.xgb, self.lgbm]
        
        base_model_predictions = [model.predict(self.x) for model in models]

        # Stack the predictions and use a meta-model
        stacked_X = np.vstack(base_model_predictions).T
        meta_model = LinearRegression()
        meta_model.fit(stacked_X, self.y)

        # Optimize meta-model weights using cross-validation
#         stacked_predictions = np.vstack([model.predict(self.x_test) for model in models]).T
        best_weights = cross_val_predict(meta_model, stacked_X, self.y, cv=5)
        
        y_pred = (
            best_weights[0] * self.lr.predict(self.pca_x) +
            best_weights[1] * self.rfr.predict(self.x) +
            best_weights[2] * self.xgb.predict(self.x) +
            best_weights[3] * self.lgbm.predict(self.x)
)
        
        
        
        
    
    
        
                    